In [ ]:
!pip install openai
!pip install langchain

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.5) # openai_api_key=OPENAI_API_KEY
llm("我饿了")

PromptTemplate: 给通用型LLMs的输入

In [ ]:
from langchain import PromptTemplate

prompt = PromptTemplate(input_variables=[], template="请给我讲个笑话")
prompt.format()

带参数的PromptTemplate

In [ ]:
prompt = PromptTemplate(input_variables=["adjective"], template="请给我讲个{adjective}的笑话.")
prompt.format(adjective="有趣")

In [ ]:
multiple_input_prompt = PromptTemplate(
    input_variables=["adjective", "content"], 
    template="请给我讲个关于{content}的{adjective}的笑话."
)
multiple_input_prompt.format(adjective="有趣", content="猪八戒")

In [ ]:
print(llm(multiple_input_prompt.format(adjective="灰色幽默", content="猪八戒")))

few_shot_prompt_template: 给定examples，让llm理解examples的语义并按照同样的格式输出

In [ ]:
prompt = """The following are exerpts from conversations with an AI
assistant. The assistant is speaking like a famous movie character.

User: Where is New York?
AI: """

print(llm(prompt))

In [ ]:
examples = [
    {
        "query": "What sports do you play?",
        "answer": "Soccer, I play."
    }, {
        "query": "Where is this?",
        "answer": "My home, this is."
    }, {
        "query": "Are you leaving now?",
        "answer": "Stay and help you, I will."
    }, {
        "query": "How are you feeling?",
        "answer": "Great I feel."
    }
]

In [ ]:
example_template = """
Human: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

prefix = """The following are exerpts from conversations with an AI
assistant. The assistant is speaking like a famous movie character.

Here are some examples:
"""

suffix = """
Human: {query}
AI: """

from langchain import FewShotPromptTemplate
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

print(few_shot_prompt_template.format(query="Where is New York?"))

In [ ]:
print(openai(few_shot_prompt_template.format(query="Where is New York?")))

example_selector: token有限，为避免最终的prompts超出token限制，对example进行部分选择

In [ ]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_template = """
Human: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=30
)

In [ ]:
few_shot_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)
print(few_shot_prompt_template.format(query="Where is New York?"))

ChatOpenAI: 用于聊天的LLM，对比通用型LLMs的区别:
1. 请求时可以传递多条消息: list[message]
2. 每条消息可以有: role, content, name
Role包含: Ai, User, System
对应4种Message: ChatMessage，SystemMessage，AIMessage, HumanMessage
对应4种PromptTemplate: ChatPromptTemplate，SystemMessagePromptTemplate，AIMessagePromptTemplate，HumanMessagePromptTemplate

In [ ]:
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(temperature=0)

In [ ]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage, ChatMessage

# 返回json格式字符串,按照每项参数拆分,不要说明和解释信息
message = [
    SystemMessage(content="返回json格式,按照每项参数拆分,不要说明和解释信息"),
    HumanMessage(content="告诉我特斯拉model Y汽车的尺寸参数"),
]
print(chat(message))

In [7]:
system_template = "你是一个把{input}翻译成{output}的助手"
system_template_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template = "{text}"
human_template_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_template_prompt, human_template_prompt])
message = chat_prompt.format_prompt(input='英语', output='中文', text='hello')
print(message)

NameError: name 'SystemMessagePromptTemplate' is not defined

In [ ]:
print(chat(message.to_messages()))

输出选择器(Output Parsers): 语言模型的输出为文本，很多时候需要获取比文本更加结构化的信息，这就是输出选择器的作用
比如上面的: “返回json格式字符串,按照每项参数拆分,不要说明和解释信息” 这个提示会让LLM返回json数据

CommaSeparatedListOutputParser: 逗号分割，列表输出解析器

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions() # 提示信息，提示LLM应该返回什么格式的数据
prompt = PromptTemplate(
    template="List 3 main-stream {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)
   

In [ ]:
print(format_instructions)

In [ ]:
llm = OpenAI(temperature=0)
input_prompt = prompt.format(subject="music styles")
output = llm(input_prompt)
print(output)

In [ ]:
output_parser.parse(output)

StructuredOutputParser: 结构化类型输出解析器

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

response_schemas = [
    ResponseSchema(name="answer", description="answer to the human's question"),
    ResponseSchema(name="source", description="source used to answer the human's question, should be a website.")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

In [ ]:
input_prompt = prompt.format_prompt(question="what are the ingredients of milk?")
output = llm(input_prompt.to_string())

print(output)

In [ ]:
json_data = output_parser.parse(output)
print(json_data['answer'])